# 알바몬 크롤러

In [1]:
#-*- coding:utf-8 -*-

from bs4 import BeautifulSoup
import urllib.request
import csv
import datetime

### 한페이지에 알바 50개 보여주는 알바몬 url주소 하나를 넣으면  
### [알바 이름, 알바 설명, 알바지역, 급여종류(시급일급 등), 급여, 노동시간, 각 알바의 상세 주소] 를 뽑아주는 함수

In [8]:
def alba_info_get(a_url):
    
    with urllib.request.urlopen(a_url) as url:  
        doc = url.read() #페이지에 있는 내용 가져와서 string 형식으로 저장해줌
    alba_doc = BeautifulSoup(doc, "html.parser")
    
    if len(alba_doc.find_all('tbody')) != 1:
        tbody = alba_doc.find_all('tbody')[1]
    else:
        tbody = alba_doc.find_all('tbody')[0]
    
    
    tlist = tbody.find_all('tr')
    
    alba_info_onepage = []
    base_url = "http://www.albamon.com/"

    for alba in tlist:
        a_name = alba.find('p', class_='cName').text
        a_juso = base_url + str(alba.a).split("\"")[1]
        a_text = alba.find('p', class_='cTit').text
        
        not_area = ['재택근무', '전국']
        not_area_bool = False
        
        for n in not_area:
            area = alba.find('td', class_ = 'area').text.replace("스크랩", "").strip()
            if (n in area) & (not_area_bool == False):
                a_area = [n, n]    
                not_area_bool = True
            else:
                a_area = alba.find_all('td')[0].text.replace("스크랩", "").strip().split(" ") # [시도, 구]
#                 a_area_B = a_area[0]  #Big 단위
#                 a_area_S = a_area[1]  #Small 단위

        if alba.find('td', class_ = "pay").text != "교육 · 연수생":
            a_paytype = str(alba.find('img')).split("\"")[1]
            a_paywon = int(alba.find('td', class_= 'pay').text.replace(",", "").replace("원", ""))
            a_worktime = alba.find_all('td')[-2].text.strip()
        else:
            a_paytype = "교육 · 연수생"
            a_paywon = 0
            a_worktime = ""

        alba_info_onepage.append([a_name, a_text, a_area, a_paytype, a_paywon, a_juso])
    return alba_info_onepage

In [9]:
# 잘 되는지 test
test = alba_info_get("http://www.albamon.com/list/gi/mon_icon_list.asp?itype=11&ps=50")
test

[['송파 장지 서울복합물류',
  '내일가능#당일지급#152,300원#초보가능#하루가능#야간풀',
  ['서울', '광진구'],
  '일급',
  152300,
  'http://www.albamon.com//recruit/view/gi?AL_GI_No=67283499&amp;mj_stat=0&amp;optgf='],
 ['송파 장지 서울복합물류',
  '내일가능#당일지급#152,300원#초보가능#하루가능#야간풀',
  ['서울', '강북구'],
  '일급',
  152300,
  'http://www.albamon.com//recruit/view/gi?AL_GI_No=67283498&amp;mj_stat=0&amp;optgf='],
 ['송파 장지 서울복합물류',
  '내일가능#당일지급#152,300원#초보가능#하루가능#야간풀',
  ['서울', '송파구'],
  '일급',
  152300,
  'http://www.albamon.com//recruit/view/gi?AL_GI_No=67283497&amp;mj_stat=0&amp;optgf='],
 ['레고코리아',
  '[레고/단기] 이마트 경산점 레고 판촉(일급 8만)',
  ['대구', '전체'],
  '일급',
  80000,
  'http://www.albamon.com//recruit/view/gi?AL_GI_No=67281131&amp;mj_stat=0&amp;optgf='],
 ['굿프랜드로지스틱스',
  '당일지급/10만원/주5일근무/한달단기/물류센터 상품 분류',
  ['경기', '성남시'],
  '일급',
  100000,
  'http://www.albamon.com//recruit/view/gi?AL_GI_No=67275043&amp;mj_stat=0&amp;optgf='],
 ['쿠팡풀필먼트서비스 - 직영C팀',
  '[쿠팡]광주 HUB 급구/주간조/직영C팀/단기/상차 분류',
  ['서울', '강동구'],
  '일급',
  72560,
  'http://www.albamon.co

### 1~page limit(지정된 변수)까지 페이지를 돌면서 각 페이지의 알바정보(50개)를 저장해주는 함수

##### 한 페이지 돌 때 마다 1초씩 쉬어야 악성코드로 판별되지 않고 잘 수집됨

In [10]:
import time

#nobar = "http://www.albamon.com/list/gi/mon_icon_list.asp?itype=11&ps=50&sExcChk=y"

def alba_page_iter(page_limit):
    page_n = 1
    base_url_top = "http://www.albamon.com/list/gi/mon_icon_list.asp?page="
    base_url_bottom = "&itype=11&ps=50"
#     nobar = "&sExcChk=y"

    alba_info_whole = []
    
    while page_n < page_limit:
        search_url = base_url_top + str(page_n) + base_url_bottom #+ nobar
        print(search_url)
        a_list = alba_info_get(search_url)
        for alba in a_list:
            alba_info_whole.append(alba)            
        page_n += 1
        time.sleep(1)


    
    return alba_info_whole
        
    
    

# 아래에서는 300페이지만큼 구해보았다. = 300 x 50개 정도 나올 예정 (250번째 페이지 이후부터는 내용이없기도함)

In [11]:
alba_list = alba_page_iter(300)

http://www.albamon.com/list/gi/mon_icon_list.asp?page=1&itype=11&ps=50
http://www.albamon.com/list/gi/mon_icon_list.asp?page=2&itype=11&ps=50
http://www.albamon.com/list/gi/mon_icon_list.asp?page=3&itype=11&ps=50
http://www.albamon.com/list/gi/mon_icon_list.asp?page=4&itype=11&ps=50
http://www.albamon.com/list/gi/mon_icon_list.asp?page=5&itype=11&ps=50
http://www.albamon.com/list/gi/mon_icon_list.asp?page=6&itype=11&ps=50
http://www.albamon.com/list/gi/mon_icon_list.asp?page=7&itype=11&ps=50
http://www.albamon.com/list/gi/mon_icon_list.asp?page=8&itype=11&ps=50
http://www.albamon.com/list/gi/mon_icon_list.asp?page=9&itype=11&ps=50
http://www.albamon.com/list/gi/mon_icon_list.asp?page=10&itype=11&ps=50
http://www.albamon.com/list/gi/mon_icon_list.asp?page=11&itype=11&ps=50
http://www.albamon.com/list/gi/mon_icon_list.asp?page=12&itype=11&ps=50
http://www.albamon.com/list/gi/mon_icon_list.asp?page=13&itype=11&ps=50
http://www.albamon.com/list/gi/mon_icon_list.asp?page=14&itype=11&ps=50
h

http://www.albamon.com/list/gi/mon_icon_list.asp?page=115&itype=11&ps=50
http://www.albamon.com/list/gi/mon_icon_list.asp?page=116&itype=11&ps=50
http://www.albamon.com/list/gi/mon_icon_list.asp?page=117&itype=11&ps=50
http://www.albamon.com/list/gi/mon_icon_list.asp?page=118&itype=11&ps=50
http://www.albamon.com/list/gi/mon_icon_list.asp?page=119&itype=11&ps=50
http://www.albamon.com/list/gi/mon_icon_list.asp?page=120&itype=11&ps=50
http://www.albamon.com/list/gi/mon_icon_list.asp?page=121&itype=11&ps=50
http://www.albamon.com/list/gi/mon_icon_list.asp?page=122&itype=11&ps=50
http://www.albamon.com/list/gi/mon_icon_list.asp?page=123&itype=11&ps=50
http://www.albamon.com/list/gi/mon_icon_list.asp?page=124&itype=11&ps=50
http://www.albamon.com/list/gi/mon_icon_list.asp?page=125&itype=11&ps=50
http://www.albamon.com/list/gi/mon_icon_list.asp?page=126&itype=11&ps=50
http://www.albamon.com/list/gi/mon_icon_list.asp?page=127&itype=11&ps=50
http://www.albamon.com/list/gi/mon_icon_list.asp?pa

http://www.albamon.com/list/gi/mon_icon_list.asp?page=228&itype=11&ps=50
http://www.albamon.com/list/gi/mon_icon_list.asp?page=229&itype=11&ps=50
http://www.albamon.com/list/gi/mon_icon_list.asp?page=230&itype=11&ps=50
http://www.albamon.com/list/gi/mon_icon_list.asp?page=231&itype=11&ps=50
http://www.albamon.com/list/gi/mon_icon_list.asp?page=232&itype=11&ps=50
http://www.albamon.com/list/gi/mon_icon_list.asp?page=233&itype=11&ps=50
http://www.albamon.com/list/gi/mon_icon_list.asp?page=234&itype=11&ps=50
http://www.albamon.com/list/gi/mon_icon_list.asp?page=235&itype=11&ps=50
http://www.albamon.com/list/gi/mon_icon_list.asp?page=236&itype=11&ps=50
http://www.albamon.com/list/gi/mon_icon_list.asp?page=237&itype=11&ps=50
http://www.albamon.com/list/gi/mon_icon_list.asp?page=238&itype=11&ps=50
http://www.albamon.com/list/gi/mon_icon_list.asp?page=239&itype=11&ps=50
http://www.albamon.com/list/gi/mon_icon_list.asp?page=240&itype=11&ps=50
http://www.albamon.com/list/gi/mon_icon_list.asp?pa

In [13]:
len(alba_list)

13730

In [12]:
# 재택근무나 전국의 경우 형태를 다른 위치정보와 동일하게 ['재택근무', '재택근무']로 바꿔줄 필요가 있음. 


for i in range(len(alba_list)):
    if len(alba_list[i][2]) == 1:
        alba_list[i][2].append(alba_list[i][2][0])
        

## 마지막 csv파일로 저장


In [16]:
import csv
 
f = open('alba_info_1215.csv', 'w', encoding='utf-8-sig', newline='')
wr = csv.writer(f)
wr.writerow(['알바명', '알바설명', '지역', '지역세부', '급여종류', '금액', '주소'])

a = alba_list
for i in range(len(a)):
    wr.writerow([a[i][0], a[i][1], a[i][2][0], a[i][2][1], a[i][3],  a[i][4], a[i][5]]) 

    

f.close()